### In this project, we use the kernelized version of Perceptron algorithm to create our prediction fot the given dataset. Due to the high time complexity of this algorithm, the process is time consuming.

In [1]:
import pandas as pd
import numpy as np

# Loading The Dataset

In [4]:
data = pd.read_csv('your_dataset.csv')

# Preparing data:
1. Seperating features and labels
2. Normalizing the dataset

In [7]:
def DataNormalize(inputData):
    mean = np.mean(inputData, axis = 0)
    std = np.std(inputData, axis = 0)
    normalizedData = (inputData - mean) / std
    
    return normalizedData

def DataShuffle(dataSize):
    indices = np.arange(dataSize)
    np.random.shuffle(indices)
    
    return indices

def DataSplit(inputData, outputData):
    # Define the split ratio
    trainRatio = 0.8
    trainSize = int(len(inputData) * trainRatio)
    
    # Split the data
    trainData = inputData[:trainSize]
    trainLabel = outputData[:trainSize]
    
    testData = inputData[trainSize:]
    testLabel = outputData[trainSize:]
    
    return trainData, trainLabel, testData, testLabel

def zeroOneLoss(trueLabels, predLabels):
    return np.sum(trueLabels != predLabels) / len(trueLabels)

In [9]:
X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values

X_normalized = DataNormalize(X)

# Shuffle the data
indices = DataShuffle(X.shape[0])
X_normalized = X_normalized[indices]
Y = Y[indices]
    
    
# Split the data
trainData, trainLabel, testData, testLabel = DataSplit(X_normalized, Y)

# Kernel Perceptron Implementation 

In [12]:
class KernelPerceptron:
    def __init__(self, kernel='gaussian', degree=3, gamma=1.0, coef0=1.0, max_iter=100):
        self.kernel = kernel
        self.degree = degree
        self.gamma = gamma
        self.coef0 = coef0
        self.max_iter = max_iter
        self.support_vectors = []  # To store indices of support vectors
        self.alphas = []
        self.X_train = None
        self.y_train = None
        self.kernel_matrix = None

    def _gaussian_kernel(self, X, Y):
        return np.exp(-self.gamma * np.linalg.norm(X[:, np.newaxis] - Y, axis=2) ** 2)

    def _polynomial_kernel(self, X, Y):
        return (self.gamma * np.dot(X, Y.T) + self.coef0) ** self.degree

    def _compute_kernel(self, X, Y):
        if self.kernel == 'gaussian':
            return self._gaussian_kernel(X, Y)
        elif self.kernel == 'polynomial':
            return self._polynomial_kernel(X, Y)
        else:
            raise ValueError(f"Unknown kernel type: {self.kernel}")

    def _precompute_kernel_matrix(self, X):
        self.kernel_matrix = self._compute_kernel(X, X)

    def _compute_single_prediction(self, x):
        if not self.support_vectors:
            return -1
        else:
            sv_indices = np.array(self.support_vectors)
            X_sv = self.X_train[sv_indices]
            K = self._compute_kernel(X_sv, x[np.newaxis, :])
            return np.sign(np.dot(self.alphas, self.y_train[sv_indices] * K.ravel()))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X_train = X
        self.y_train = y
        self._precompute_kernel_matrix(X)
        
        for iteration in range(self.max_iter):
            no_errors = True
            for t in range(n_samples):
                prediction = self._compute_single_prediction(X[t])
                if prediction != y[t]:
                    self.support_vectors.append(t)
                    self.alphas.append(1)  # Use alpha for weight adjustment
                    no_errors = False
            
            if no_errors:
                print(f"Stopping early at iteration {iteration}")
                break

    def predict(self, X):
        if not self.support_vectors:
            return np.zeros(X.shape[0])
        
        predictions = []
        for t in range(X.shape[0]):
            prediction = self._compute_single_prediction(X[t])
            predictions.append(prediction)
        
        return np.array(predictions)


# Cross Validation Implementation

In [15]:
# Grid Search using Cross Validateion
def crossValScore(X, y, params, k = 5):
    fold_size = len(X) // k
    accuracies = []
    
    for i in range(k):
        
        X_test = X[i * fold_size:(i + 1) * fold_size]
        y_test = y[i * fold_size:(i + 1) * fold_size]
        X_train = np.concatenate((X[:i * fold_size], X[(i + 1) * fold_size:]), axis=0)
        y_train = np.concatenate((y[:i * fold_size], y[(i + 1) * fold_size:]), axis=0)

        kernelPerceptron = KernelPerceptron(kernel = params['kernel'], degree =  params['degree'], gamma = params['gamma'], coef0 = params['coef0'], max_iter =  params['max_iter'])
        
        kernelPerceptron.fit(X_train, y_train)
        
        predictions = kernelPerceptron.predict(X_test)
        accuracy = zeroOneLoss(predictions, y_test)
        
        accuracies.append(accuracy)
        
    return np.mean(accuracies)

# Tuning Hyperparameters

### In order to speed up the process, we use parallel programming and use all existing CPU threads for each set of hyperparameters.

In [29]:
from joblib import Parallel, delayed

# Gaussian kernel parameters:
gamma_values = [0.1, 1, 10]
max_iter_values = [20, 30, 40]

# Polynomial kernel parameters
degree_values = [2, 3, 4]
coef0_values = [1, 5, 10]

inputData = trainData[: int(0.2 * len(trainData))]
outputData = trainLabel[: int(0.2 * len(trainLabel))]

def evaluate_params(params):
    mean_loss = crossValScore(inputData, outputData, params, k=3)
    return params, mean_loss

# Gaussian kernel
print("Tuning Hyperparameters for Gaussian Kernel: \n")
best_gaussian_loss = 2
best_gaussian_params = {}

# Generate parameter combinations for Gaussian kernel
gaussian_params_list = [{'kernel': 'gaussian', 'degree': 3, 'gamma': gamma, 'coef0': 1.0, 'max_iter': iters}
                        for gamma in gamma_values for iters in max_iter_values]

# Evaluate all parameter combinations in parallel
results = Parallel(n_jobs=-1)(delayed(evaluate_params)(params) for params in gaussian_params_list)

for params, mean_loss in results:
    print(f"gamma: {params['gamma']}, n_iters: {params['max_iter']}, Cross Validation Loss:{mean_loss}")
    if mean_loss < best_gaussian_loss:
        best_gaussian_loss = mean_loss
        best_gaussian_params = params

print("\nBest Hyperparameters:\n")
print("Gaussian: ")
print(f"\t gamma: {best_gaussian_params['gamma']}")
print(f"\t max_iters: {best_gaussian_params['max_iter']}")
print(f"\t Best Cross-Validation loss: {best_gaussian_loss}")

------------------------------------------------------------
Tuning Hyperparameters for Gaussian Kernel: 

gamma: 0.1, n_iters: 20, Cross Validation Loss:0.07629768605378362
gamma: 0.1, n_iters: 30, Cross Validation Loss:0.07191994996873045
gamma: 0.1, n_iters: 40, Cross Validation Loss:0.06879299562226392
gamma: 1, n_iters: 20, Cross Validation Loss:0.17073170731707318
gamma: 1, n_iters: 30, Cross Validation Loss:0.17073170731707318
gamma: 1, n_iters: 40, Cross Validation Loss:0.17073170731707318
gamma: 10, n_iters: 20, Cross Validation Loss:0.23076923076923075
gamma: 10, n_iters: 30, Cross Validation Loss:0.23076923076923075
gamma: 10, n_iters: 40, Cross Validation Loss:0.23076923076923075

Best Hyperparameters:

Gaussian: 
	 gamma: 0.1
	 max_iters: 40
	 Best Cross-Validation loss: 0.06879299562226392


In [31]:
params = best_gaussian_params
kernelPerceptron = KernelPerceptron(kernel = params['kernel'], degree = params['degree'], gamma = params['gamma'], coef0 = params['coef0'], max_iter =  params['max_iter'])
kernelPerceptron.fit(trainData, trainLabel)
preds = kernelPerceptron.predict(testData)
zeroOneLoss(preds, testLabel)

0.0395